# Transfer Learning

Steps to follow:

- Load the existing model
- Use full model or select the first layers manually using `model.layers`
- Freeze the gotten model/layers
- Create your full network (Function API may be easier)
    - Adding pre_processing (tf models already come with pre_processing layer)
    - Adding last layers
- Train
- Unfreeze
- Train again

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
import os
import matplotlib.pyplot as plt
import numpy as np

_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')

train_dir = os.path.join(PATH, 'train')
validation_dir = os.path.join(PATH, 'validation')

BATCH_SIZE = 32
IMG_SIZE = (160, 160)

train_dataset = image_dataset_from_directory(train_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

Found 2000 files belonging to 2 classes.


In [7]:
validation_dataset = image_dataset_from_directory(validation_dir, batch_size=BATCH_SIZE, image_size=IMG_SIZE)

Found 1000 files belonging to 2 classes.


In [8]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [9]:
print('Number of train batches: %d' % tf.data.experimental.cardinality(train_dataset))
print('Number of validation batches: %d' % tf.data.experimental.cardinality(validation_dataset))
print('Number of test batches: %d' % tf.data.experimental.cardinality(test_dataset))

Number of validation batches: 26
Number of test batches: 6


## Rescale images

MobileNetV2 has images ranging `[-1, 1]`

In [14]:
for img, labels in train_dataset.take(1):
    print(np.max(img.numpy()))    # Our dataset seems to be from [0, 255]

255.0


In [55]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

## Get MobileNetV2

In [48]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')

In [49]:
base_model.summary()

Model: "mobilenetv2_1.00_160"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         input_4[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 80, 80, 32)   0           bn_Conv1[0][0]                   
_______________________________________________________________________________

In [50]:
base_model.layers

In [51]:
for i in range(0, len(base_model.layers)):
    base_model.layers[i].trainable = False
# base_model.trainable = False

In [52]:
base_model.layers[1].trainable

False

## Create network
We are going to use Functional API

In [57]:
inputs = tf.keras.Input(shape=(160, 160, 3))
x = preprocess_input(inputs)
x = base_model(x, training=False)
# With this parameter there is no need to set the trainable layers to False right?
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1)(x)

model = tf.keras.Model(inputs, outputs)

In [58]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [59]:
len(model.trainable_variables)

2

In [60]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281  

## Train

In [61]:
initial_epochs = 10

loss0, accuracy0 = model.evaluate(validation_dataset)

26/26 [==============================] - 5s 143ms/step - loss: 0.7490 - accuracy: 0.5050


In [62]:
history = model.fit(train_dataset, epochs=initial_epochs, validation_data=validation_dataset)

Epoch 1/10
63/63 [==============================] - 15s 219ms/step - loss: 0.6025 - accuracy: 0.6355 - val_loss: 0.4738 - val_accuracy: 0.7153
Epoch 2/10
63/63 [==============================] - 14s 219ms/step - loss: 0.4115 - accuracy: 0.7795 - val_loss: 0.3303 - val_accuracy: 0.8688
Epoch 3/10
63/63 [==============================] - 14s 219ms/step - loss: 0.3045 - accuracy: 0.8700 - val_loss: 0.2563 - val_accuracy: 0.9121
Epoch 4/10
63/63 [==============================] - 14s 222ms/step - loss: 0.2383 - accuracy: 0.9165 - val_loss: 0.2070 - val_accuracy: 0.9307
Epoch 5/10
63/63 [==============================] - 15s 228ms/step - loss: 0.2021 - accuracy: 0.9305 - val_loss: 0.1738 - val_accuracy: 0.9468
Epoch 6/10
63/63 [==============================] - 15s 227ms/step - loss: 0.1726 - accuracy: 0.9405 - val_loss: 0.1575 - val_accuracy: 0.9493
Epoch 7/10
63/63 [==============================] - 15s 228ms/step - loss: 0.1511 - accuracy: 0.9495 - val_loss: 0.1421 - val_accuracy: 0.9579

## Unfreeze an keep training

In [63]:
base_model.trainable = True
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf.math.truediv_1 (TFOpLambd (None, 160, 160, 3)       0         
_________________________________________________________________
tf.math.subtract_1 (TFOpLamb (None, 160, 160, 3)       0         
_________________________________________________________________
mobilenetv2_1.00_160 (Functi (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 1281  

In [64]:
# One must always compile again
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [65]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=validation_dataset)

Epoch 10/20
63/63 [==============================] - 44s 643ms/step - loss: 0.0758 - accuracy: 0.9780 - val_loss: 0.0545 - val_accuracy: 0.9802
Epoch 11/20
63/63 [==============================] - 40s 629ms/step - loss: 0.0267 - accuracy: 0.9923 - val_loss: 0.0461 - val_accuracy: 0.9827
Epoch 12/20
63/63 [==============================] - 39s 621ms/step - loss: 0.0102 - accuracy: 0.9972 - val_loss: 0.0452 - val_accuracy: 0.9851
Epoch 13/20
63/63 [==============================] - 40s 625ms/step - loss: 0.0055 - accuracy: 0.9984 - val_loss: 0.0509 - val_accuracy: 0.9814
Epoch 14/20
63/63 [==============================] - 39s 620ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 0.0579 - val_accuracy: 0.9827
Epoch 15/20
63/63 [==============================] - 39s 619ms/step - loss: 3.0542e-04 - accuracy: 1.0000 - val_loss: 0.0570 - val_accuracy: 0.9851
Epoch 16/20
63/63 [==============================] - 42s 659ms/step - loss: 9.6659e-04 - accuracy: 1.0000 - val_loss: 0.0585 - val_a